# 潛力股預測系統 - 資料探索

本筆記本用於探索台灣股票系統中的財務資料，為機器學習模型準備特徵。

## 目標
- 了解資料庫中的資料結構和品質
- 探索各種財務指標的分布
- 分析目標變數的特性
- 識別潛在的特徵工程機會

In [ ]:
# 導入必要的套件
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 設置中文字體
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 導入自定義模組
from src.utils.database import DatabaseManager
from src.features.feature_engineering import FeatureEngineer
from src.features.target_generator import TargetGenerator

# 設置顯示選項
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ 套件導入完成")

## 1. 資料庫連接和基本資訊

In [ ]:
# 初始化資料庫管理器
db_manager = DatabaseManager()

# 獲取股票清單
stock_list = db_manager.get_stock_list(exclude_patterns=['00'])
print(f"📊 股票總數: {len(stock_list)}")
print(f"📈 上市股票: {len(stock_list[stock_list['market'] == 'TWSE'])}")
print(f"📉 上櫃股票: {len(stock_list[stock_list['market'] == 'TPEX'])}")

# 顯示股票清單樣本
print("\n📋 股票清單樣本:")
stock_list.head(10)

## 2. 資料表資訊探索

In [ ]:
# 檢查各個資料表的資訊
tables = ['stock_prices', 'monthly_revenues', 'financial_statements', 'balance_sheets', 'cash_flow_statements']

for table in tables:
    try:
        info = db_manager.get_table_info(table)
        print(f"\n📊 {table}:")
        print(f"   資料筆數: {info['row_count']:,}")
        print(f"   欄位數量: {len(info['columns'])}")
        
        # 顯示欄位資訊
        columns_df = pd.DataFrame(info['columns'])
        print("   主要欄位:")
        for _, col in columns_df.head(5).iterrows():
            print(f"     - {col['name']} ({col['type']})")
            
    except Exception as e:
        print(f"❌ 無法獲取 {table} 資訊: {e}")

## 3. 樣本股票資料探索

In [ ]:
# 選擇幾個代表性股票進行探索
sample_stocks = ['2330', '2317', '2454', '3008', '6505']  # 台積電、鴻海、聯發科、大立光、台塑化
end_date = '2024-12-31'
start_date = '2022-01-01'

print("🔍 探索樣本股票資料...")

for stock_id in sample_stocks:
    try:
        # 獲取股價資料
        prices = db_manager.get_stock_prices(stock_id, start_date, end_date)
        
        # 獲取月營收資料
        revenues = db_manager.get_monthly_revenue(stock_id, start_date, end_date)
        
        # 獲取財務報表資料
        financials = db_manager.get_financial_statements(stock_id, start_date, end_date)
        
        stock_name = stock_list[stock_list['stock_id'] == stock_id]['stock_name'].iloc[0] if len(stock_list[stock_list['stock_id'] == stock_id]) > 0 else stock_id
        
        print(f"\n📈 {stock_id} ({stock_name}):")
        print(f"   股價資料: {len(prices)} 筆")
        print(f"   月營收資料: {len(revenues)} 筆")
        print(f"   財務報表資料: {len(financials)} 筆")
        
        if not prices.empty:
            latest_price = prices.iloc[-1]
            print(f"   最新股價: {latest_price['close_price']:.2f} ({latest_price['date']})")
            
    except Exception as e:
        print(f"❌ 股票 {stock_id} 資料探索失敗: {e}")

## 4. 特徵生成測試

In [ ]:
# 測試特徵工程
feature_engineer = FeatureEngineer(db_manager)

print("🔧 測試特徵生成...")

# 為樣本股票生成特徵
sample_features = []

for stock_id in sample_stocks[:3]:  # 只測試前3個股票
    try:
        features = feature_engineer.generate_features(stock_id, '2024-06-30')
        if not features.empty:
            sample_features.append(features)
            print(f"✅ {stock_id}: 生成 {len(features.columns)-2} 個特徵")  # -2 for stock_id and feature_date
        else:
            print(f"⚠️ {stock_id}: 無法生成特徵")
    except Exception as e:
        print(f"❌ {stock_id}: 特徵生成失敗 - {e}")

if sample_features:
    # 合併所有特徵
    all_features = pd.concat(sample_features, ignore_index=True)
    
    print(f"\n📊 特徵統計:")
    print(f"   樣本數: {len(all_features)}")
    print(f"   特徵數: {len(all_features.columns)-2}")
    
    # 顯示特徵樣本
    feature_cols = [col for col in all_features.columns if col not in ['stock_id', 'feature_date']]
    print(f"\n🔍 特徵樣本 (前10個):")
    for col in feature_cols[:10]:
        print(f"   {col}: {all_features[col].describe()['mean']:.4f} (平均值)")
else:
    print("❌ 沒有成功生成任何特徵")

## 5. 目標變數分析

In [ ]:
# 測試目標變數生成
target_generator = TargetGenerator(db_manager)

print("🎯 測試目標變數生成...")

# 生成樣本目標變數
feature_dates = ['2024-03-31', '2024-06-30', '2024-09-30']
sample_targets = []

for stock_id in sample_stocks[:3]:
    try:
        targets = target_generator.generate_targets(stock_id, feature_dates)
        if not targets.empty:
            sample_targets.append(targets)
            print(f"✅ {stock_id}: 生成 {len(targets)} 個目標變數")
    except Exception as e:
        print(f"❌ {stock_id}: 目標變數生成失敗 - {e}")

if sample_targets:
    # 合併所有目標變數
    all_targets = pd.concat(sample_targets, ignore_index=True)
    
    print(f"\n📊 目標變數統計:")
    print(f"   樣本數: {len(all_targets)}")
    print(f"   正樣本數: {(all_targets['target'] == 1).sum()}")
    print(f"   正樣本比例: {all_targets['target'].mean():.2%}")
    print(f"   平均最大報酬率: {all_targets['max_return'].mean():.2%}")
    
    # 分析目標變數分布
    analysis = target_generator.analyze_target_distribution(all_targets)
    print(f"\n📈 目標變數分析:")
    for key, value in analysis.items():
        if isinstance(value, dict):
            print(f"   {key}:")
            for sub_key, sub_value in value.items():
                print(f"     {sub_key}: {sub_value:.4f}")
        else:
            print(f"   {key}: {value}")
else:
    print("❌ 沒有成功生成任何目標變數")

## 6. 資料品質檢查

In [ ]:
# 檢查資料品質
print("🔍 資料品質檢查...")

if sample_features:
    features_df = pd.concat(sample_features, ignore_index=True)
    
    # 檢查缺失值
    missing_stats = features_df.isnull().sum()
    missing_features = missing_stats[missing_stats > 0]
    
    print(f"\n📊 缺失值統計:")
    if len(missing_features) > 0:
        print(f"   有缺失值的特徵數: {len(missing_features)}")
        print("   主要缺失特徵:")
        for feature, count in missing_features.head(10).items():
            print(f"     {feature}: {count}/{len(features_df)} ({count/len(features_df):.1%})")
    else:
        print("   ✅ 沒有缺失值")
    
    # 檢查異常值
    numeric_features = features_df.select_dtypes(include=[np.number]).columns
    numeric_features = [col for col in numeric_features if col not in ['stock_id']]
    
    print(f"\n📊 數值特徵統計:")
    print(f"   數值特徵數: {len(numeric_features)}")
    
    if len(numeric_features) > 0:
        # 計算基本統計
        stats = features_df[numeric_features].describe()
        print("\n   基本統計 (前5個特徵):")
        print(stats.iloc[:, :5].round(4))
        
        # 檢查極值
        print("\n   極值檢查:")
        for col in numeric_features[:5]:
            values = features_df[col].dropna()
            if len(values) > 0:
                q1, q3 = values.quantile([0.25, 0.75])
                iqr = q3 - q1
                outliers = values[(values < q1 - 1.5*iqr) | (values > q3 + 1.5*iqr)]
                print(f"     {col}: {len(outliers)} 個異常值 ({len(outliers)/len(values):.1%})")

print("\n✅ 資料探索完成！")